In [ ]:
from tensorflow.keras.layers import Input,Conv1D,GlobalAveragePooling1D,LSTM,Dense,Reshape,BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation, Embedding, Flatten, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
import gc
import matplotlib.pyplot as plt
from scipy import stats
from time import sleep
import seaborn
import keras

from tqdm import tqdm
from random import choices

from pympler.tracker import SummaryTracker
tracker = SummaryTracker()


import keras_tuner as kt

physical_devices = tf.config.list_physical_devices('GPU')
try:
          tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
          # Invalid device or cannot modify virtual devices once initialized.
    pass

In [ ]:
gc.collect()

In [ ]:
class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1,callbacks=None):
        global hist, model
        val_losses = []
        for fold, (train_indices, test_indices) in enumerate(splits):
            if fold != 6:
                gc.collect()
                tf.keras.backend.clear_session()
                
                print(f'Beginning Training on Fold: {fold}')

                x_train, x_test = X[train_indices], X[test_indices]
                y_trn, y_tst = y[train_indices], y[test_indices]
                if len(x_train) < 2:
                    x_train = x_train[0]
                    x_test = x_test[0]
                if len(y_trn) < 2:
                    y_trn = y_trn[0]
                    y_tst = y_tst[0]

                model = self.hypermodel.build(trial.hyperparameters)
                gc.collect()
                hist = model.fit(x_train,y_trn,validation_data=(x_test,y_tst), epochs=epochs, batch_size=batch_size,
                                 callbacks=callbacks)

                del x_train, y_trn, x_test, y_tst
                gc.collect()
                val_losses.append([hist.history[k][-1] for k in hist.history])
                tf.keras.backend.clear_session()
                gc.collect()
            else:
                pass

            
        
        val_losses = np.asarray(val_losses)
        print('VAL_LOSSES: ', val_losses)
        losses = {}
        print('LOSSES: ', losses)
        print('model: ', model)
        for i, k in enumerate(hist.history.keys()):
            losses[k] = np.mean(val_losses[:,i])
            
        print(losses)
        
        self.oracle.update_trial(trial.trial_id, losses)

In [ ]:
def correlationMetric(x, y, axis=-2):
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.math.reduce_sum(x, axis=axis)
    ysum = tf.math.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.math.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.math.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.math.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return tf.constant(1.0, dtype=x.dtype) - corr

def correlationLoss(x,y, axis=-2):
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.math.reduce_sum(x, axis=axis)
    ysum = tf.math.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xsqsum = tf.math.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    ysqsum = tf.math.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.math.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.math.sqrt(xsqsum * ysqsum)
  # absdif = tmean(tf.abs(x - y), axis=axis) / tf.sqrt(yvar)
    sqdif = tf.math.reduce_sum(tf.math.squared_difference(x, y), axis=axis) / n / tf.sqrt(ysqsum / n)
    return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (0.01 * sqdif)) , dtype=tf.float32 )


In [ ]:
import os
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate, Layer
debug_flag = int(os.environ.get('KERAS_ATTENTION_DEBUG', 0))

class Attention(object if debug_flag else Layer):
    def __init__(self, units = 128, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.units = units
    
    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        with K.name_scope(self.name if not debug_flag else 'attention'):
            self.attention_score_vec = Dense(input_dim, use_bias=False, name = 'attention_score_vec')
            self.h_t = Lambda(lambda x: x[:, -1, :], output_shape=(input_dim,), name = 'last_hidden_state')
            self.attention_score = Dot(axes=[1,2], name = 'attention_score')
            self.attention_weight = Activation('softmax', name = 'attention_weight')
            self.context_vector = Dot(axes = [1,1], name = 'context_vector')
            self.attention_output = Concatenate(name = 'attention_output')
            self.attention_vector = Dense(self.units, use_bias=False, activation = 'tanh', name = 'attention_vector')
            if not debug_flag:
                super(Attention, self).build(input_shape)
            
    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
    def __call__(self, inputs, training = None, **kwargs):
        if debug_flag:
            return self.call(inputs, training, **kwargs)
        else:
            return super(Attention, self).__call__(inputs, training, **kwargs)
    
    def call(self, inputs, training = None, **kwargs):
        if debug_flag:
            self.build(inputs.shape)
        score_first_part = self.attention_score_vec(inputs)
        h_t = self.h_t(inputs)
        score = self.attention_score([h_t, score_first_part])
        attention_weights = self.attention_weight(score)
        context_vector = self.context_vector([inputs, attention_weights])
        pre_activation = self.attention_output([context_vector, h_t])
        attention_vector = self.attention_vector(pre_activation)
        
        return attention_vector
    def get_config(self):
        config = super(Attention, self).get_config()
        config.update({'units':self.units})
        return config

In [ ]:
class LiteModel:
    
    @classmethod
    def from_file(cls, model_path):
        return LiteModel(tf.lite.Interpreter(model_path=model_path))
    
    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))
    
    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out
    
    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]


In [ ]:

import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
#---pulling in data---
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
ft_remove = ['f_8','f_22','f_55','f_62','f_63','f_74','f_99','f_108','f_124','f_142','f_145','f_150','f_153','f_170','f_175','f_182','f_200','f_225','f_241','f_272']
df.drop(ft_remove, inplace = True, axis = 1)
FEATURES = [f for f in df.columns if 'f' in f]
TARGET = ['target']
DATE = ['time_id']

In [ ]:
X = df[FEATURES].values
time = df[DATE].values
y = df[TARGET].values
del df
gc.collect()

In [ ]:
ENCODING = True
if ENCODING:
    XEncodingData1 = X[:785352]
    XEncodingData2 = X[785352:1570704]
    XEncodingData3 = X[1570704:2356056]
    XEncodingData4 = X[2356056:]
    yEncodingData1 = y[:785352]
    yEncodingData2 = y[785352:1570704]
    yEncodingData3 = y[1570704:2356056]
    yEncodingData4 = y[2356056:]
    XEnocdedData = [(XEncodingData1,yEncodingData1), (XEncodingData2,yEncodingData2), (XEncodingData3,yEncodingData3), (XEncodingData4,yEncodingData4)]

In [ ]:
from keras.layers import Input, Conv1D, LeakyReLU, MaxPool1D, CuDNNLSTM, Bidirectional, TimeDistributed, Dense, Reshape
from keras.layers import UpSampling2D, Conv2DTranspose

def AutoEncoderModel(input_shape, output_shape):
    inputLayer = Input(shape = (1,input_shape))
    encode = Conv1D(500, kernel_size = 10, strides = 2, padding = 'same', activation = 'tanh')(inputLayer)
    encode = LeakyReLU()(encode)
    encode = MaxPool1D(pool_size=5, padding='same')(encode)
    encode = Bidirectional(CuDNNLSTM(250, return_sequences=True), merge_mode='concat')(encode)
    encode = LeakyReLU()(encode)
    encode = Bidirectional(CuDNNLSTM(75, return_sequences=True), merge_mode='concat')(encode)
    encode = LeakyReLU(name = 'latent')(encode)
    decode = TimeDistributed(Dense(500))(encode)
    decode = LeakyReLU()(decode)
    decode = Reshape((-1,1,500))(decode)
    decode = UpSampling2D((1, 1), name="upsampling")(decode)
    decode = Conv2DTranspose(input_shape,(10,1), padding='same',name = 'conv2dtranspose')(decode)
    output = Reshape((-1,input_shape))(decode)
    
    encodeFlatten = Flatten()(encode)
    
    
    y = Dense(150, activation = 'swish')(output)
    y = Dense(output_shape, activation='linear',name = 'y')(y)
    
    encodeFlatten = Dense(75)(encodeFlatten)
    
    encoder = Model(inputs = inputLayer, outputs = encodeFlatten)
    decoder = Model(inputs = inputLayer, outputs = output)
    autoencoder = Model(inputs = inputLayer, outputs = [output, y])
    
    autoencoder.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'mse', metrics='mse')
    
    return autoencoder,encoder,decoder
    
    

In [ ]:
autoencoder, encoder, decoder = AutoEncoderModel(XEnocdedData[0][0].shape[1], XEnocdedData[0][1].shape[1])

In [ ]:
encoder.summary()

In [ ]:
TRAINING_AE = False
Encoders = []
if TRAINING_AE:
    for i, data in enumerate(XEnocdedData):
        inputX = np.reshape(data[0],(-1,1,280))
        inputy = np.reshape(data[1],(-1,1,1))
        autoencoder.fit((inputX),(inputX,inputy), epochs=25, validation_split=0.2,batch_size = 20000, callbacks= [EarlyStopping('val_mse', patience = 10, restore_best_weights = True)])
        encoder.save(f'./encoder_model_{i}.h5')
        gc.collect()
        tf.keras.backend.clear_session()
else:
    for i in range(4):
        mdl = keras.models.load_model(f'../input/encoderweigtsubq/encoder_model_{i}.h5')
        Encoders.append(mdl)
encoder.trainable = False

In [ ]:
Encoders[0]._name = "encoder1"
Encoders[1]._name = "encoder2"
Encoders[2]._name = "encoder3"
Encoders[3]._name = "encoder4"

In [ ]:
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
def DNNmodel (params, ftInputShape):
    
    ft_input = Input(shape=ftInputShape, name="FEATURE_INPUT")
    
    reshapeft_input = Reshape((1,ftInputShape))(ft_input)
    
    enc1 = Encoders[0](reshapeft_input)
    enc1 = Flatten()(enc1)
    enc2 = Encoders[1](reshapeft_input)
    enc2 = Flatten()(enc2)
    enc3 = Encoders[2](reshapeft_input)
    enc3 = Flatten()(enc3)
    enc4 = Encoders[3](reshapeft_input)
    enc4 = Flatten()(enc4)
    
    x = Concatenate()([enc1,enc2,enc3,enc4,ft_input])
    x = BatchNormalization()(x)
    x = Dropout(params.Float('init_dropout',0.0, 0.5))(x)
    
    for i in range(params.Int('num_layers',5,8)):
        x0 = Dense(params.Int('Dense_layer_1',500,1000), kernel_initializer = tf.keras.initializers.GlorotNormal())(x)
        x0 = BatchNormalization()(x0)
        x0 = Dense(params.Int('Dense_layer_2',250,500), kernel_initializer = tf.keras.initializers.GlorotNormal())(x0)
        x0 = BatchNormalization()(x0)
        x0 = Dense(params.Int('Dense_layer_3',25,250))(x0)
        x0 = BatchNormalization()(x0)
        x0 = Dropout(params.Float(f'dropout{i}', 0.0, 0.5))(x0)
    
    
    
    
    

    input_lstm = Dense(256,  kernel_initializer = tf.keras.initializers.GlorotNormal())(ft_input)
    reshape_input = Reshape((1,256))(input_lstm)
    
    x1 = BatchNormalization()(reshape_input)
    x1 = LSTM(128, return_sequences = True, recurrent_dropout=0.3, activation='relu')(x1)
    x1 = LSTM(32, dropout=0.1, return_sequences=True, activation='relu')(x1)
    a1 = Attention(units = 16)(x1)
    

    final = Concatenate()([a1,x0])
    final = Dense(params.Int('final_1', 128,256))(final)
    final = BatchNormalization()(final)
    final = Dense(params.Int('final_1', 10,75))(final)
    final = BatchNormalization()(final)
    final = Dropout(params.Float('Final_dropout',0.0,0.5))(final)
    final = BatchNormalization()(final)
    x = Dense(1, activation = 'linear')(final)
    
    
    model = Model(inputs = [ft_input],outputs = [x])
    model.compile(optimizer = Adam(learning_rate=params.Float('Learning_rate',0.01,0.00001)), loss = 'mse',  metrics = [correlationMetric,'mse'])
    
    return model

In [ ]:
X.shape

In [ ]:
tf.random.set_seed(2020)
model_fn = lambda params: DNNmodel(params, 280)
tuner = CVTuner(hypermodel=model_fn, oracle = kt.oracles.BayesianOptimizationOracle(objective=kt.Objective('val_mse', direction = 'min'),max_trials=5, allow_new_entries=True), overwrite=True)

In [ ]:
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.pyplot as plt
    
# this is code slightly modified from the sklearn docs here:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
# gkf = PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=50, max_train_group_size=500, max_test_group_size=150)
# time = np.reshape(time, (len(X),))
# fig, ax = plt.subplots(figsize = (20,20))
# plot_cv_indices(gkf, X, y, time, ax, 5, lw=60);

In [ ]:
TRAINING_TUNER = True
FOLDS = 5
if TRAINING_TUNER:
    gkf = PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=50, max_train_group_size=500, max_test_group_size=150)
    #gkf = GroupKFold(n_splits=FOLDS)
    time = np.reshape(time, (len(X),))
    splits = list(gkf.split(X,y,groups = time))
    tuner.search(X, y, splits = splits, callbacks = [EarlyStopping('val_mse', mode = 'min', patience = 4)], epochs = 20, batch_size = 10000)
    params = tuner.get_best_hyperparameters(1)[0]
    pd.to_pickle(params, f'./best_params.pkl')

In [ ]:
def inference(model_finetuned, model_trained_on_folds , ds):
    Xint = ds[FEATURES].values
    y_finetuned = []
    y_trainedOnFolds = []
    for model in model_finetuned:
        y_predFine = model.predict(Xint)
        y_finetuned.append(y_predFine)
        
    for model in model_trained_on_folds:
        y_predFold = model.predict(Xint)
        y_trainedOnFolds.append(y_predFold)

#     print(y_finetuned)
#     print(y_trainedOnFolds)
    yTotal = 0.7*np.mean(y_finetuned, axis = 0) + 0.3*np.mean(y_trainedOnFolds, axis = 0)
    
    return yTotal

In [ ]:
# simpleModel = []
# fineTunedModel = []
# for fold in range(25):
#     if fold == 1 or fold == 10 or fold == 21:
#         print(f'../input/testparams/model_{fold}.hdf5')
#         model.load_weights(f'../input/testparams/model_{fold}.hdf5')
#         mdl = LiteModel.from_keras_model(model)
#         simpleModel.append(mdl)

In [ ]:
# for fold in range(25):
#     if fold == 1 or fold == 10 or fold == 21:
#         print(f'../input/testparams/model_{fold}.hdf5')
#         model.load_weights(f'../input/testparams/model_{fold}_finetune.hdf5')
#         mdl = LiteModel.from_keras_model(model)
#         fineTunedModel.append(mdl)

In [ ]:
# def weighted_average(a):
#     w = []
#     n = len(a)
#     for j in range(1, n + 1):
#         j = 2 if j == 1 else j
#         w.append(1 / (2**(n + 1 - j)))
#     print(w)
#     return np.average(a, weights = w, axis = 0)

In [ ]:
# testInput = np.reshape(X[1], (1,300))
# t = inference(fineTunedModel, simpleModel, testInput)
# #out = []
# # for model in simpleModel:
# #     pred = model.predict(testInput)
# #     out.append(pred)

# # print(np.mean(out, axis = 0))

In [ ]:
# gkf = PurgedGroupTimeSeriesSplit(n_splits=FOLDS,group_gap=100, max_train_group_size=100, max_test_group_size=50)
# #gkf = GroupKFold(n_splits=FOLDS)
# time = np.reshape(time, (len(X),))
# splits = list(gkf.split(X,y,groups = time))
# for fold, (train_indices, test_indices) in enumerate(splits):
#     if fold == 1 or fold == 10 or fold == 21:
#         model = model_fn(params)
#         X_train, X_test = X[train_indices], X[test_indices]
#         y_train, y_test = y[train_indices], y[test_indices]
#         model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 5000, callbacks=[EarlyStopping('val_mse',mode='min',patience=6,restore_best_weights=True)])
#         model.save_weights(f'./model_{fold}.hdf5')
#         model.compile(Adam(params.get('Learning_rate')/100),loss='mse')
#         model.fit(X_test,y_test,epochs=5,batch_size=5000)
#         model.save_weights(f'./model_{fold}_finetune.hdf5')

In [ ]:
TRAININGMODEL = False
if TRAININGMODEL:
    for fold, (train_indices, test_indices) in enumerate(splits):
        if fold != 6:
            gc.collect()
            tf.keras.backend.clear_session()
            model = model_fn(params)
            X_train, X_test = X[train_indices], X[test_indices]
            y_train, y_test = y[train_indices], y[test_indices]
            model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 5000, callbacks=[EarlyStopping('val_mse',mode='min',patience=6,restore_best_weights=True)])
            model.save(f'./model_{fold}.h5')
            gc.collect()
            del X_train, y_train
            model.compile(Adam(params.get('Learning_rate')/100),loss='mse')
            model.fit(X_test,y_test,epochs=15,batch_size=5000)
            model.save(f'./model_{fold}_finetune.h5')
            del X_test,y_test
        else:
            pass
else:
    params = pd.read_pickle('../input/paramubq/best_params.pkl')
    model = model_fn(params)
    tf.keras.backend.clear_session()
    model.summary()
    model_trained_on_folds = []
    model_finetuned = []
    for fold in range(0,FOLDS):
        model.load_weights(f'../input/drubqmodelweights/model_{fold}.hdf5')
        mdl = LiteModel.from_keras_model(model)
        model_trained_on_folds.append(mdl)
    
    for fold in range(0,FOLDS):
        model.load_weights(f'../input/drubqmodelweights/model_{fold}_finetune.hdf5')
        mdl = LiteModel.from_keras_model(model)
        model_finetuned.append(mdl)

In [ ]:
TRAINING_FULL_MODEL = False
if TRAINING_FULL_MODEL:
    model.fit(X,y, epochs = 100, batch_size = 10000, callbacks=[EarlyStopping('mse',mode='min',patience=6,restore_best_weights=True)])
    model.save('./modelFullData.h5')
else:
    model.load_weights('../input/drubqmodelweights/modelFullData.hdf5')
    fullModel = LiteModel.from_keras_model(model)

In [ ]:
UBQ = False
if UBQ:
    
    import ubiquant
    #api = uu.API(test_df)
    env = ubiquant.make_env()
    iter_test = env.iter_test()

    for (test_df, sample_prediction_df) in iter_test:
        sample_prediction_df['target'] = inference(model_finetuned, model_trained_on_folds, test_df)
        env.predict(sample_prediction_df) 